# Open AI Function Calling

In [ ]:
%pip install openai

In [6]:
import os.path
import requests

# Download einer Beispiel-Speisekarte, welche bereits in strukturiertem Python-Code vorliegt.
# Könnte man später durch einen Datei- oder Webseitenimport ersetzen.

MENU_DEMO_URL = "https://raw.githubusercontent.com/TAS-AG/TAS.OpenAI.Workshop/refs/heads/main/data/speisekarte_pizza_sample.py"
MENU_MICELLOS_URL = "https://github.com/TAS-AG/TAS.OpenAI.Workshop/blob/main/data/speisekarte_micellos_leipzig.pdf"
MENU_ATRIUM_URL = "https://raw.githubusercontent.com/TAS-AG/TAS.OpenAI.Workshop/refs/heads/main/data/bistro_im_atrium_wochenkarte.html"

download_urls = [MENU_DEMO_URL, MENU_MICELLOS_URL, MENU_ATRIUM_URL]

for url in download_urls:
    response = requests.get(url)
    filename = os.path.basename(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

from speisekarte_pizza_sample import MENU

In [ ]:
import base64

with open("speisekarte_micellos_leipzig.pdf", "rb") as f:
    data = f.read()

speisekarte_micellos_base64_string = base64.b64encode(data).decode("utf-8")

In [ ]:
from openai import OpenAI
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')
openai_client = OpenAI(api_key = openai_api_key)

In [ ]:
import json

user_input = "Ich habe Lust auf was mit Gorgonzola."

expected_response_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",  #Erwarteter Rückgabetyp ist ein Array von Objekten
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

response = openai_client.responses.create(
    model = "gpt-4o",
    input = [
        {
            "role": "developer",
            "content": [
                {
					"type": "input_text",
					"text": "Du bist ein digitaler Kellner. Wähle den passenden Eintrag aus der Speisekarte aus."
				}
            ]
        },
        {
            "role": "user",
            "content":  [
               {
                    "type": "input_file",
                    "filename": "speisekarte_micellos_leipzig.pdf",
                    "file_data": f"data:application/pdf;base64,{speisekarte_micellos_base64_string}",
                },
                {
                    "type": "input_text",
                    "text": f"{user_input}",
                }
			]
		}
    ],
    # Gewünschtes Ausgabeformat wird API-Aufruf übergeben
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": expected_response_schema,
            "strict": True
        }
    }
)
recommendations = json.loads(response.output_text)["items"]

for item in recommendations:
    print(item)
